#### 3. Assume that you want to find quadratic functions of $\mathbf{x}$, i.e., $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$. How would you formulate this in a deep network?

# Solution
**Key Insight:** We can apply arbitrary operations on the feature set (like multiplying them) to increase the expressiveness of our system.

We define one layer that computes all products $x_ix_j$ for $1 \leq i <= j < n$. We define a dense layer that is linear to compute the terms $b + \sum_i w_i x_i$. We also create a dense layer with linear activation without a bias term that computes the expression $\sum_{i \leq j} w_{ij} x_ix_j$ based on the previously computed feature interactiosn ($x_ix_j$). Then we add the ouput of the terms.

Note that if we define $\underline{w} = (w_1, w_2, \dots, w_n)$ and
$$
W = \begin{pmatrix}
w_{11} & 0 & \cdots & 0 \\
w_{21} & w_{22} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
w_{n1} & w_{n2} & \cdots & w_{nn}
\end{pmatrix}
$$
then the function can be expressed as follows
$$
f(x) = b + \underline{w} \cdot \underline{x} + \underline{x}^t W \underline{x}.
$$
We can flatten $W$ by embedding it into $\mathbb{R}^{\frac{n(n + 1)}{2}}$ (for a proof see below) and then take the dot product of the weight vector.

## Proof and details of embedding
For the dimensionality note that $W \in \mathbb{R}^{n \times n}$. Removing the diagonal leaves us with $n^2 - n$ elements, half of that is $(n^2 - n) / 2$, which is exactly the number of elements we have to remove because we have a lower triangular matrix. So the number of elements we actually want to have is $n^2 - \left( \frac{n^2 - n}{2} \right) = \frac{n(n + 1)}{2}$.

We want to define an explicit map $\widetilde{\phi} : \R­_{\Delta^u}^{n \times n} \hookrightarrow \R^{n(n+1)/2}$, which reduces to finding an isomorphism
$$
\phi : \{(i, j) \in \mathbb{N} : 1 \leq i \leq j \leq n\} \rightarrow \{k \in \mathbb{N} : 1 \leq k \leq n(n+1)/2\}
$$

The $kth$ row of the matrix has $k$ entries, so to get the $jth$ element of the $kth$ row ($1 \leq j \leq k$) is $T_k + j$ where $T_{k - 1}$ is the $kth$ triangle number, which we can compute using Gauß's little theorem (ger. kleiner Gauß)
$$
T_{k-1} = \sum_{i = 1}^{k - 1} i = \frac{(k - 1)k}{2}
$$
so that
$$
\phi^{-1}(T_{k - 1} + j) = (k, j).
$$

Note that the dimension of the vector space in the range is exactly $T_n$.

## PyTorch code implementation
This is purely for future reference to recall the concepts discussed here, it's generated by Claude3 and untested.

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor

class QuadraticFeatures(nn.Module):
    """
    Module to create quadratic feature interactions from input data.

    Parameters:
    num_features (int): Number of input features.
    """
    def __init__(self, num_features: int):
        super(QuadraticFeatures, self).__init__()
        self.num_interactions = num_features * (num_features + 1) // 2
        self.weights = nn.Parameter(torch.randn(self.num_interactions))

    def forward(self, x: Tensor) -> Tensor:
        """
        Computes the quadratic feature interactions (x_i * x_j) and applies the weights.
        """
        # Could probably vectorize this computation for efficiency, something like
        # x @ x.T probably would work, as x in R^n and x.T in R^(1 x n)
        # so their product is in R^(n x n) and consist of all x_i * x_j
        interactions = []
        for i in range(x.size(1)):
            for j in range(i, x.size(1)):
                # Compute the pairwise product of features i and j (x_i * x_j)
                interactions.append(x[:, i] * x[:, j])

        interactions = torch.stack(interactions, dim=1)
        return torch.matmul(interactions, self.weights)

class MyQuadraticModel(nn.Module):
    """
    Model that combines linear and quadratic relationships between input features and target variable.
    """
    def __init__(self, num_features: int):
        super(MyQuadraticModel, self).__init__()
        self.linear = nn.Linear(num_features, 1)
        self.quadratic_features = QuadraticFeatures(num_features)
        self.final_linear = nn.Linear(2, 1, bias=False)

    def forward(self, x: Tensor) -> Tensor:
        """
        Computes the model output by combining linear and quadratic outputs.
        """
        linear_output = self.linear(x)
        quadratic_output = self.quadratic_features(x)
        combined = torch.cat((linear_output, quadratic_output.unsqueeze(1)), dim=1)
        return self.final_linear(combined)

num_features = 3
model = MyQuadraticModel(num_features)
input_x = torch.randn(10, num_features)
output = model(input_x)
print(output)

```python
tensor([[ 0.1154],
        [ 0.4561],
        [ 0.3887],
        [ 0.4996],
        [ 0.6570],
        [-0.2477],
        [ 0.5168],
        [ 0.2347],
        [ 0.3938],
        [ 0.8847]], grad_fn=<MmBackward0>)
```